### UC Berkeley, MICS, W202-Cryptography
### Week 02 Breakout 5

### Speeding up RSA encryption and decryption using Short Public Exponents and the Chinese Remainder Theorem


In this breakout, we will see 2 ways we can speed up RSA encryption and decryption.  For public keys, we can use Short Public Exponents which give a trememdous speed up.  For private keys, we can use the Chinese Remainder Theorem with p and q giving a modest increase in speed.


### Using Short Public Exponents to speed up public key operations

In RSA, e and d are both exponents.  When their values are small, we call this "short exponents", and when their values are large, we call this "long exponents".  As we saw in our breakout last week, if e is a short exponent, d will be a long exponent, and vice versa.

We NEVER, EVER want a private key to be a short exponent. This is fairly obvious. An attacker could use a simple loop starting at 3 until they discover the other exponent.  

Since short exponents should only be used for public keys, we typically call them "short public exponents."

In the case of key establishment, the short public exponent would be e.

In the case of digital signatures, the short public exponent would be d.  Our algorithms for finding e and d are interchangeable, we can use the same algorithm setting d to the short public exponent and find e using the Extended Euclidean Algorithm to solve Bezout's Identity. (essentially we can swap e and d)

The most common short public exponents are:
* 3,                binary: 11, number of operations in the fast powering algorithm: 2
* 17,               binary: 10001, number of operations in the fast powering algorithm: 5
* 65537 = 2^16 + 1, binary: 10000000000000001, number of operations in the fast powering algorithm: 17

There are published papers which demonstrate that 65537 is the shortest public exponent that is safe to use.  65537 seems to be the most widely used short public exponent, especially for digital signatures.  Most certificate authorities use it.  65537 is a Fermat Prime, actually the largest Fermat Prime.  It is designated as F4, because it is the Fermat Prime with n = 4 in Fermat's 2^2^n.

If we don't need a speed up, we normally generate a large odd number of similar size betweem n/2 to n, which gives both e and d a similar large size.





### To understand better how this works, revisit the breakout from last week:

04_1_rsa.ipynb

Set the bit value for n to 4096.

Re-execute the jupyter notebook.

Examine the values for e and d when e is generated as a random number with similar bit length to n.

There is a code cell with instruction for setting the value of e and then verifying it.  Follow these instructions to set e = 65537. Re-execute the cells to set and validate e through finding d.

Examine the value for d and compare it to the value for d found above.

Time permitting (you may want to do the next part first), try different bit sizes for n (64, 128, 256, 512, 1024, 2048, 4096) and try different short public exponents for e (3, 17, 65537)

### Using the Chinese Remainder Theorem to speed up private key operations

Short public exponents only speed up the public exponent. So, how can we speed up the private exponent?  The answer is to use the Chinese Remainder Theorem (CRT) with p and q instead of using n directly.

There are some considerations to think about:
* It's the only known speedup for private key encryption
* The CRT will only give us a modest speed up compared to the speed up with the short public exponent
* The private key needs to include p and q for this to work. Do we want to include p and q in the private key?

The speed is increased in two ways:
* The fast powering algorithm is used individually with p and q and the mod values.  The smaller mod values make the result of each step much smaller for the next step. This more than makes up for the process to combine the two results at the end.
* Four of the values used in calculation based on d, p, q can be re-computed.

At the high level, the algorithm works like this:
* Certain values are not dependent on the ciphertext c, so we can precompute them
* Ciphertext c is converted to CRT Notation using (mod p) and (mod q) instead of (mod n)
* Ciphertext c in CRT Notation is raised to it's power for p in (mod p) and it's power for q in (mod q)
* Ciphertext c is converted into (mod n) from the CRT Notation using (mod p) and (mod q)


In [1]:
from sage.all import *

In [2]:
def my_print_number(label, x):
    "prints a number in decimal, number of digits, hex, number of bits"
    
    print ("\n", label, '\n')
    print ("decimal:", "{:,}".format(x), "\n")
    print ("number of digits:", x.ndigits(), "\n")
    print ("hex:", x.hex(), "\n")
    print ("number of bits:", x.nbits(), "\n")

In [3]:
# n is of size 4096 bits
# n = pq

# d is the private key
# p and q are the factors of n 
# c is the ciphertext of the encrypted message 

# n, c are public
# d, p, q are secret

n = 0xb88347ce4256b45e8297fdb60e5d2ea0e5512f7bce0bfa568228d6cd5cc2063304bbeb3b2f4f9f9bc0be456dc15f7456cbd0549e25dde0fda07e8c3cc977fd08c829bab0d72337ba1d7e1d5ff41ab46f8a650c45a9f0689bfb731fd6cae0734a7cf5aef64ce4a014dee89d268d3617eef7ec452688549b037aea55ceca9016d5f85a46366090189b6bf0348aeefe52058a4d6ee29558dde93797b5017473ea0a4d9afdf0e3e8c110ab4963746941cb5522dcd2457edd3eab3086980054d4cf6c5efc175ccbab31df54ecbb19e04ed01d8d9957e81ba5e9dabf9134f96578264f7d0c743721a7fff977f1c691eac6e70ec4d9d6bf850d150fd74f911e6a12c1e5661f72455d08f4e3d8e09483d954f7b33973c1091808dd7ee1b8b583b3cfc858d8f7ea55c56fecbe65c4cb22235ca60acc7176ae09a7376c47da82196f444959c7edbea2eaeb42ee305eefbefa17c012ad66a23b584764cb101e21cdae4281ec284dda36432320b5847e49132bdd70d03cb9516aa0498d0d289f94507de1915f7d5cbfa466cd14636765a48cf2a6527d6ef1a6ec20fb13a8e5a97d298f29c784c2d4e9ce649409cb645612370271ee7f291b056205d1453c5770b4449ec47431e1fa8acbfa85e5fb0a90cfd8cf65f3f6f32f5265f915be8dd31ebd6f73767c2de41cd6d8e19a695779c627e48e98f8785c3af22f0552ea635be1e171e4c1c27d

p = 0xdf9426b6498d33b565bf191c61a9a92dd891119448407fb4015f34e3471252d3bf097483d86c1237f1418e5950e419e5fea17eeb2a46bf5f661aea4fae00093deae3a6f83f2525ce30580f922cf3d228b3f52af22f6f15f7f4ee89b27dd03f24ccca153f8028c047019d145cddb696a298bc7c1dd32219c784fb4f452e6bfa9f7291b1895ebb0a2f9569d2488bd22338e0dac4b7f2ffec3b1a1fabfd3eb447a6ce2eb91b38ef1a4a075b1d0b7294fdc90f5314cae9756f4965fce9220f3616feec9856440728bd72b4a6e653edbfee0cdee4215e38e0bc91d906853507551dc2cf0c588e0dccdac99b93abcfc01a657abc16b17c04819bc06686f2803e4f3a7b

q = 0xd344e5a0897bf129a281db91ae283ae1aecb8108bd894a6a3fc2c2eee68903b358f22f2eb20b4be3335004b61b819d158ab98f2f12c0979159179ae07e389873681dc32ea78be6ecdd824ba93f326d60cc769e3e0ec7708523de41a370b5af65e7131639e58041af74b9b6a71e481515caf012fe08e45bb085acafc399188243940d62ae07625f0dd555b7ab748f76e25d920a7203014590771f33ef940b5f5a2a2005ea62c4901c9791078de657e58cb747641da2bd708fb0b30946b81b348775168e3d793a8d7b77a586eb9f73aec93c09469deb104d653179abb0ee8e692268320061239587b15a9a1fd8a99ab2117f87cccfd2db1aa2e266d9e134984167

d = 0x1bdc85c8fbe049da353afd96aca79e9ffcfdd1c4391ac1b6fb35eaf0312c2f849a2a642eed87063d5f3c33e6523d2a8427f34c130253890079a9b065fd388e30826ef2339112f6311f5e00c98addd52be4179820cc4fbb407f9f766fd31f8e37579726ecb7df184439059a1e45d3b0f110cdc04a966dd67f11362ea6419daa1ae32dfec7ae03b9faf6e5edcd766194385102b0ea7620c44bbb788b9678040b9379a3a832a7298f093a796d357ece9d1ae8aa4ddc71e4c9632ae02d892d749b5d4d523c994326580de9348bfca4e97cdefcec78306b6abb096a9d1c6a512d1fb4609f60bfafccd8cb6587766cb17e15fb8750eaed91aa0a305a78f28e11e4d4a2c42e1e30a766b78fe0a1c7a1a3514d4c5a6952f8966c3474a834eb018fa9c921ee65892f9080e4ce7b407da214d1de6505b1f05082b91dabd0509e4c4364607f303c2a4fe8806ed16bf91d74eba02ed9d2b3ee7f6f0cc5f351adbd0114ddac3539a427a0ac64f99442cc2894ee9f2bc0c8bfc2a7061781fdb47237c84213f8df8421d456b2ae54008d348103982cc1b83afceb600d69e2ddb031e92a108c682392847e19b3a4780e1f272f43cbaada9d876215f4582ba1b1d7de24321dbd29ee81c94e094c9a66783e0ed021ad8a78aec70d75c56460259c9395713b15da0c50104d1bfd7d6bab360bcd253141d50a627e05031fd0b33d4ca34cd079e0679bd5

c = 0xa44863dd29b0fbefc4c953f99cc45eb5d26b20e0de4090a4e787de5006bf22d492103b072a3ba945209a8a8fe21f7f5b12a87a5e77267bb47091c888a484bedd7ad3400284f8bd093128bf85db6428fca36130515ac8d5aed93fed089122b1b0b1cdd423876fcf397e6b4b5627fff0aa8c37c489f150ea691aea38b40c0221b76a2f05f0147e0c8314b67764e24c03870d6cc1f98201a8b3efd6c6a0fca2986b710197a61884156afc12b066f1df811d85e4ef0d1df4ebb52f28c7ebc602d0d1bd10ac864e3892405ecc316793ca0bf237a89b3d92159567e9566eef66cc97f367dd6e2a384002df2a9e51c9476e486134b587301be5a36b490a76c038e8a1aa474a1e6b63f94af6e19ba78633ee021ec67d9d5f03432e0278d78605c523767b7f56b75e264209996a4db3db56c21b46b8bf868a7214398debdbfd872c1ff7142eda96b3b4239bfc01a84f9b48116303896b2d895c6573c718ca07b5aa18243b1292e7a51a815c862e0a8e59155dbebc25ac244bc27c08a539cd6f47f64a8c4d5f91e1adffe6a6c88eb48a091782c02bd0aea8ac60b72d304b331a7b7c567d14b5a792e8a5dabd379f51f3a8e95d3327b8251ff0ed8fa4052fab54097026c5d93691a42738deb541d0edaf7543155e1cd63e6454320ebf01005a4064c658417d91fd998ff62dd83cf6ca60b1ba1171a780704d80be5ee51bb648fd1777b8d487


In [4]:
my_print_number("n", n)


 n 

decimal: 752,746,608,814,198,088,289,207,379,883,644,715,023,101,046,605,388,535,435,955,829,590,402,471,881,811,120,771,067,313,452,807,040,505,314,285,592,891,830,427,965,816,468,154,584,197,215,795,175,694,673,787,308,746,348,993,937,857,575,178,704,434,482,925,597,335,126,871,316,841,414,779,537,849,102,222,389,269,697,592,630,908,271,320,307,809,133,486,284,298,793,979,569,854,507,994,672,503,564,077,335,972,183,075,637,201,620,786,132,140,840,096,775,226,252,162,506,028,412,111,145,321,187,390,196,224,163,848,190,625,458,840,047,345,579,742,348,488,161,182,886,939,707,802,417,030,209,200,683,477,967,611,921,581,964,005,310,392,064,381,959,253,456,619,954,067,740,386,370,834,513,414,735,719,742,017,316,125,479,731,583,349,661,421,197,514,981,836,471,414,817,347,592,070,331,855,593,557,418,738,778,202,715,467,538,719,219,275,197,108,527,130,292,910,574,601,228,441,358,512,575,119,102,981,167,107,425,415,490,375,137,246,283,053,991,368,931,368,715,231,955,666,007,148,559,066,2

In [5]:
my_print_number("p", p)


 p 

decimal: 28,224,198,096,349,620,167,798,430,032,760,432,929,205,324,750,469,622,888,396,124,490,393,404,143,917,304,365,387,893,875,853,073,594,452,697,311,223,735,671,063,770,753,324,952,999,960,370,066,894,097,965,626,094,158,381,596,958,186,933,717,775,412,271,078,905,054,401,722,972,394,333,815,985,282,585,010,431,777,884,055,096,798,918,870,432,860,191,285,411,705,175,813,400,803,475,596,142,680,453,114,338,547,693,008,661,487,709,611,767,970,757,965,102,669,638,331,043,649,600,243,663,327,694,718,702,149,242,465,284,548,949,068,312,553,382,855,629,852,816,093,146,337,328,264,916,950,448,368,074,078,877,473,668,387,331,635,823,102,108,301,240,801,464,918,844,024,560,892,658,353,611,511,507,559,557,595,173,710,589,074,741,430,781,259,572,509,831,698,083,650,014,420,617,263,355,871,365,814,091,980,241,851,489,663,611 

number of digits: 617 

hex: df9426b6498d33b565bf191c61a9a92dd891119448407fb4015f34e3471252d3bf097483d86c1237f1418e5950e419e5fea17eeb2a46bf5f661aea4fae00093deae

In [6]:
my_print_number("q", q)


 q 

decimal: 26,670,256,715,337,987,070,127,446,976,794,576,809,005,918,099,095,759,080,646,731,436,147,715,873,016,428,603,091,399,069,604,755,704,045,084,574,982,798,757,506,314,593,614,834,639,803,653,404,457,515,472,233,856,922,618,148,425,234,316,934,998,357,663,633,968,859,943,322,417,405,474,315,781,150,170,106,140,336,464,979,080,286,829,709,879,558,997,683,330,713,234,560,093,210,458,647,327,933,905,193,519,620,320,826,439,755,603,791,594,198,711,082,770,117,800,295,780,931,688,628,908,753,384,755,552,712,232,472,125,192,810,159,967,960,808,830,942,060,556,416,645,489,776,689,244,168,045,831,557,572,809,808,318,491,318,789,838,026,823,117,205,662,454,430,718,001,977,343,546,665,442,168,073,075,253,455,885,053,400,581,370,650,435,682,765,344,348,956,541,959,953,870,358,116,941,269,047,489,147,787,655,135,811,354,983 

number of digits: 617 

hex: d344e5a0897bf129a281db91ae283ae1aecb8108bd894a6a3fc2c2eee68903b358f22f2eb20b4be3335004b61b819d158ab98f2f12c0979159179ae07e389873681

In [7]:
my_print_number("d", d)


 d 

decimal: 113,664,662,136,855,701,740,132,446,381,009,921,180,193,077,822,953,168,468,024,581,898,616,222,263,347,827,696,945,432,431,601,759,429,394,838,937,795,428,092,074,311,077,954,385,520,560,706,135,683,109,289,257,838,762,326,782,347,375,731,585,743,550,016,502,847,083,436,617,100,360,126,249,661,727,105,984,337,446,782,352,684,517,762,132,633,770,396,210,625,070,790,023,938,863,379,881,596,181,971,064,470,509,735,290,184,765,531,413,527,474,475,474,378,557,487,194,977,180,603,740,463,839,951,418,513,782,615,077,391,793,457,606,817,369,337,325,773,190,936,857,809,190,641,679,280,379,170,184,792,070,180,347,231,731,491,567,662,777,849,070,097,206,928,719,213,448,583,777,541,156,796,998,175,436,036,036,436,066,933,528,084,396,076,843,157,846,075,103,779,020,943,037,272,760,259,942,699,645,369,574,778,661,867,126,621,715,606,039,966,822,683,321,063,508,818,104,688,856,490,122,573,502,188,600,609,923,863,080,674,188,937,889,526,648,148,457,212,163,503,302,071,143,608,710,604,2

In [8]:
my_print_number("c", c)


 c 

decimal: 670,215,243,621,060,340,036,540,742,961,518,085,248,958,034,078,917,044,574,735,779,190,246,219,928,739,838,942,414,544,412,030,550,311,479,530,037,773,527,019,555,224,136,783,644,809,038,559,488,612,906,741,794,730,821,471,880,651,321,679,079,099,926,667,268,859,165,545,371,116,203,643,749,111,283,539,288,569,064,231,783,905,201,845,715,476,524,268,548,123,635,159,118,246,202,774,438,100,263,700,348,498,341,515,254,665,885,762,094,083,334,826,386,197,594,031,864,193,326,345,534,329,382,964,546,678,170,598,499,701,465,158,524,644,711,365,795,728,404,722,975,350,446,629,517,941,205,080,036,202,965,410,235,332,763,759,571,570,726,817,935,788,069,274,876,546,501,488,670,947,023,396,486,286,704,403,929,886,519,787,620,232,198,902,912,790,208,856,324,152,045,793,045,465,529,270,729,935,979,967,127,528,666,672,176,796,138,371,675,668,080,480,714,790,483,971,297,448,900,700,216,944,513,372,128,640,979,300,498,211,040,714,683,031,324,288,017,033,277,085,968,998,262,484,064,615,3

In [9]:
# we can precompute the following values which we will use in our algorithm
# for future runs with the same n and d, we can reuse this

# the regular mod() function will not work with such large numbers, so we have to call the power_mod() function with exponent 1

# d_p is the d mod (p-1) residue
d_p = power_mod(d, 1, (p-1))

# d_q is the d mod (q-1) residue
d_q = power_mod(d, 1, (q-1))

# p_expression is equal to p times the multiplicitive inverse of p in (mod q)
p_expression = p * power_mod(p, -1, q)

# q_expression is equal to q times the multiplicitive inverse of q in (mod p)
q_expression = q * power_mod(q, -1, p)

All remaining calculations are dependent on the ciphertext c and cannot be pre-computed

In [10]:
# we now change the ciphertext c into it's representation using the CRT Notation

c_p = power_mod(c, 1, p)
c_q = power_mod(c, 1, q)

print ("\nciphertext c in CRT Notation:\n")
print ("<\n" + str(c_p) + "\nmod\n" + str(p) + "\n,\n" + str(c_q) + "\nmod\n" + str(q) + "\n>")


ciphertext c in CRT Notation:

<
1936675113689409799177823233917294914960516897389151964074434533225113521234408255228785634931680233113882935439001220868949744745927143688393737267231883522544394761508445766452154524698066592052036556004124373399721621657328974214950623005347792251691346469426038324254154637842048049506829270376349313613397817078673764143078602103894078277710993466369353660643692531165999471440562952350501118613714082426140861668311296624096375176316355966350818178357970298427604959076096427561201006541757603459288832438487545410876994833438863676279606735481309665894890175589713482786161668637643465366969737262737446439875
mod
282241980963496201677984300327604329292053247504696228883961244903934041439173043653878938758530735944526973112237356710637707533249529999603700668940979656260941583815969581869337177754122710789050544017229723943338159852825850104317778840550967989188704328601912854117051758134008034755961426804531143385476930086614877096117679707579651026696

In [11]:
# now we perform our exponentiations with the reduced moduli p and q instead of n

# it takes the place of the usual m = power_mod(c,d,n) and runs much faster
# but the result is in CRT Notation which have to convert back in the next cell!

m_p = power_mod(c_p, d_p, p)
m_q = power_mod(c_q, d_q, q)

In [12]:
# change <m_p mod p, m_q mod q> from CRT notation into our normal (mod n) format, which should give us our message

# if the message is 1234567890 it is correct

m = power_mod((q_expression * m_p) + (p_expression * m_q), 1, n)
m

1234567890

In [13]:
# let's compare this with just using d as the exponent

# this should also be 1234567890

m = power_mod(c,d,n)
m

1234567890